In [ ]:
pip install scikit-learn==1.3.0


  Using cached scikit_learn-1.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.9 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-tabular 1.2 requires scikit-learn<1.5.3,>=1.4.0, but you have scikit-learn 1.3.0 which is incompatible.
autogluon-core 1.2 requires scikit-learn<1.5.3,>=1.4.0, but you have scikit-learn 1.3.0 which is incompatible.
autogluon-multimodal 1.2 requires scikit-learn<1.5.3,>=1.4.0, but you have scikit-learn 1.3.0 which is incompatible.
autogluon-features 1.2 requires scikit-learn<1.5.3,>=1.4.0, but you have scikit-learn 1.3.0 whic

In [ ]:
pip install autogluon

  Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.0
    Uninstalling scikit-learn-1.3.0:
      Successfully uninstalled scikit-learn-1.3.0


In [ ]:
pip install optuna

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgbm
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from autogluon.tabular import TabularPredictor
import optuna

In [ ]:
X_train = pd.read_csv('x_train.csv')
y_train = pd.read_csv('y_train.csv')
X_val = pd.read_csv('x_val.csv')
y_val = pd.read_csv('y_val.csv')
X_test = pd.read_csv('x_test.csv')
y_test = pd.read_csv('y_test.csv')
X_pred = pd.read_csv('x_pred.csv')

In [ ]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.765098,1.302559,2.000868,-0.240127,0.947514,-0.080907,-0.296037,-0.635586,0.333628,1.321026,1.445148,1.330054,-0.212727,-0.120736,-0.092965,-0.130444
1,-0.451364,0.582372,-0.330589,-0.199755,-0.913852,-0.080907,-0.296037,-0.635586,-0.581118,-0.710543,0.621883,0.642590,-0.212727,-0.258486,-0.292874,-0.223186
2,-0.451364,-0.497907,-0.142568,-0.159480,-0.913852,-0.080907,-0.296037,0.907333,-0.581118,0.231244,-0.671818,-0.629217,-0.212727,0.517527,-0.277496,-0.159095
3,-0.451364,-0.858000,-0.518610,1.631884,-0.913852,-0.080907,-0.296037,0.907333,-1.495865,-0.172379,-0.671818,-0.801083,-0.212727,-0.614760,-0.446650,7.152335
4,-1.667826,-1.578186,-1.408575,-0.155382,-0.913852,-0.080907,-0.296037,2.450251,-0.581118,-1.127620,-0.671818,-0.835456,-0.212727,1.258201,-0.723447,-0.177736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11220,0.765098,1.662652,2.652674,-0.113853,0.947514,-0.080907,-0.296037,-0.635586,2.163120,3.231507,-0.671818,1.467546,-0.212727,0.461273,-0.200608,-0.086567
11221,-0.451364,0.222279,-0.054825,-0.169410,-0.913852,-0.080907,-0.296037,0.907333,0.333628,-1.019987,1.680367,-0.285486,-0.212727,0.427377,0.153077,-0.166459
11222,0.765098,0.582372,2.477188,0.707954,0.947514,-0.080907,-0.296037,-0.635586,2.163120,3.043150,-0.671818,0.539471,-0.212727,1.349794,2.259810,1.546075
11223,-1.667826,0.582372,1.085834,0.105819,0.947514,12.359900,5.142622,2.450251,1.248374,0.244698,1.609801,0.470724,-0.212727,1.538748,0.875825,0.422208


In [ ]:
models = [
    ('KNN', KNeighborsRegressor(), {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance'],
    }),
    ('LR', LinearRegression(), {
        'fit_intercept': [True, False]
    }),
    ('DT', DecisionTreeRegressor(random_state=42), {
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 5],
    }),
    ('RF', RandomForestRegressor(random_state=42), {
        'n_estimators': [50, 100],
        'max_depth': [10, None],
    }),
    ('XGB', XGBRegressor(random_state=42), {
        'n_estimators': [50, 100],
        'learning_rate': [0.01, 0.1],
    })
]

best_models = {}
best_rmse_scores = {}

for name, model, param_grid in models:
    grid_search = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='neg_root_mean_squared_error')
    grid_search.fit(X_train, y_train)
    best_models[name] = grid_search.best_estimator_
    best_rmse_scores[name] = -grid_search.best_score_

for name, model in best_models.items():
    print(f"Best model for {name}: {model}")
    print(f"Best RMSE score for {name}: {best_rmse_scores[name]}")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usi

Best model for KNN: KNeighborsRegressor(n_neighbors=7, weights='distance')
Best RMSE score for KNN: 150581.84029916272
Best model for LR: LinearRegression()
Best RMSE score for LR: 172186.7635628669
Best model for DT: DecisionTreeRegressor(max_depth=10, min_samples_split=5, random_state=42)
Best RMSE score for DT: 161463.12044154256
Best model for RF: RandomForestRegressor(random_state=42)
Best RMSE score for RF: 127272.58145121508
Best model for XGB: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
 

In [ ]:
predictor = TabularPredictor(label='price').fit(train_data=pd.concat([X_train, y_train], axis=1))
best_model = predictor.model_best
print("Best Model from AutoGluon:", best_model)

No path specified. Models will be saved in: "AutogluonModels/ag-20250207_112903"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.41 GB / 12.67 GB (82.1%)
Disk Space Avail:   73.12 GB / 107.72 GB (67.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions 

[1000]	valid_set's rmse: 112135


	-112003.7765	 = Validation score   (-root_mean_squared_error)
	2.88s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: LightGBM ...
	-109190.4225	 = Validation score   (-root_mean_squared_error)
	1.62s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-119293.6823	 = Validation score   (-root_mean_squared_error)
	28.45s	 = Training   runtime
	0.17s	 = Validation runtime
Fitting model: CatBoost ...
	-110078.22	 = Validation score   (-root_mean_squared_error)
	22.04s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-117518.9344	 = Validation score   (-root_mean_squared_error)
	8.47s	 = Training   runtime
	0.24s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-113108.134	 = Validation score   (-root_mean_squared_error)
	14.74s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	-110768.2227	 = Validation score   (-root_mean_squared_error)
	4.36s	 = Training   runti

Best Model from AutoGluon: WeightedEnsemble_L2


In [ ]:
leaderboard = predictor.leaderboard(silent=True)
print("AutoGluon Leaderboard:")
print(leaderboard)

AutoGluon Leaderboard:
                  model      score_val              eval_metric  \
0   WeightedEnsemble_L2 -105994.846837  root_mean_squared_error   
1              LightGBM -109190.422502  root_mean_squared_error   
2              CatBoost -110078.220002  root_mean_squared_error   
3               XGBoost -110768.222728  root_mean_squared_error   
4            LightGBMXT -112003.776521  root_mean_squared_error   
5         LightGBMLarge -112460.718932  root_mean_squared_error   
6       NeuralNetFastAI -113108.134009  root_mean_squared_error   
7         ExtraTreesMSE -117518.934363  root_mean_squared_error   
8        NeuralNetTorch -117729.633869  root_mean_squared_error   
9       RandomForestMSE -119293.682258  root_mean_squared_error   
10       KNeighborsDist -147361.510617  root_mean_squared_error   
11       KNeighborsUnif -148111.301321  root_mean_squared_error   

    pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  \
0        0.313025  136.983528

-> Trong số các thuật toán dễ cài, LightGBM đứng top 2. Nên ta sẽ chọn LightGBM.

In [ ]:
def objective(trial):
    param_grid = {
        #         "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "max_bin": trial.suggest_int("max_bin", 200, 300),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
        "random_state": 42
    }

    lgbm_model = lgbm.LGBMRegressor(**param_grid)
    lgbm_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='rmse',
        callbacks=[lgbm.early_stopping(100)]
    )
    preds = lgbm_model.predict(X_val)
    return mean_squared_error(y_val, preds, squared=False)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
best_params.update({"objective": "regression", "metric": "rmse", "random_state": 42})

best_model = lgbm.LGBMRegressor(**best_params)
best_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
    callbacks=[lgbm.early_stopping(100)]
)
preds = best_model.predict(X_test)

final_rmse = np.sqrt(mean_squared_error(y_test, preds))

print(final_rmse)

[I 2025-02-07 11:47:25,060] A new study created in memory with name: no-name-c7ac8086-da29-4ed4-b324-6a9762742652
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1096]	valid_0's rmse: 151377	valid_0's l2: 2.2915e+10


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-02-07 11:47:27,592] Trial 0 finished with value: 151377.12911633885 and parameters: {'n_estimators': 10000, 'learning_rate': 0.23865657555654138, 'num_leaves': 2500, 'max_depth': 5, 'min_data_in_leaf': 500, 'max_bin': 285, 'lambda_l1': 55, 'lambda_l2': 85, 'min_gain_to_split': 0.6462963021488205, 'bagging_fraction': 0.4, 'bagging_freq': 1, 'feature_fraction': 0.6000000000000001}. Best is trial 0 with value: 151377.12911633885.
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regressio

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 307390	valid_0's l2: 9.44887e+10
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 307390	valid_0's l2: 9.44887e+10
Training until validation scores don't improve for 100 rounds


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-02-07 11:47:28,443] Trial 3 finished with value: 138658.94400699143 and parameters: {'n_estimators': 10000, 'learning_rate': 0.1797092755432578, 'num_leaves': 1280, 'max_depth': 3, 'min_data_in_leaf': 200, 'max_bin': 209, 'lambda_l1': 35, 'lambda_l2': 5, 'min_gain_to_split': 1.9649982424152173, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.9}. Best is trial 3 with value: 138658.94400699143.
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureW

...

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2146]	valid_0's rmse: 154245	valid_0's l2: 2.37915e+10


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-02-07 11:48:46,342] Trial 48 finished with value: 154244.97032177637 and parameters: {'n_estimators': 10000, 'learning_rate': 0.1529950333830076, 'num_leaves': 1140, 'max_depth': 6, 'min_data_in_leaf': 1300, 'max_bin': 284, 'lambda_l1': 45, 'lambda_l2': 65, 'min_gain_to_split': 0.017143304385773626, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.7}. Best is trial 12 with value: 136550.5225203963.
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1316]	valid_0's rmse: 145208	valid_0's l2: 2.10854e+10


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-02-07 11:48:48,632] Trial 49 finished with value: 145208.1740507404 and parameters: {'n_estimators': 10000, 'learning_rate': 0.18263418459720157, 'num_leaves': 1360, 'max_depth': 4, 'min_data_in_leaf': 700, 'max_bin': 278, 'lambda_l1': 85, 'lambda_l2': 5, 'min_gain_to_split': 4.516940528568986, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.9}. Best is trial 12 with value: 136550.5225203963.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[309]	valid_0's rmse: 136551
169910.42668266295


In [ ]:
y_pred = best_model.predict(X_pred)

In [ ]:
df = pd.DataFrame(y_pred, columns=["price"])
df.to_csv("y_pred.csv", index=False)